## Data Cleaning

In [1]:


# import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt     
import seaborn as sns
from sklearn.model_selection import train_test_split    
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix



### Load the datasets


In [5]:
# There are 5 datasets to be loaded with the following code
prod_data = pd.read_csv(r"C:\Users\Etteba\OneDrive\Documents\DataSciencePortfolioProjects\PredPerishProducts\Amdari-Perishable-Prediction\Datasets\product_details.csv")
store_data = pd.read_csv(r"C:\Users\Etteba\OneDrive\Documents\DataSciencePortfolioProjects\PredPerishProducts\Amdari-Perishable-Prediction\Datasets\store_info.csv")
supplier_data = pd.read_csv(r"C:\Users\Etteba\OneDrive\Documents\DataSciencePortfolioProjects\PredPerishProducts\Amdari-Perishable-Prediction\Datasets\supplier_info.csv")
weather_data = pd.read_csv(r"C:\Users\Etteba\OneDrive\Documents\DataSciencePortfolioProjects\PredPerishProducts\Amdari-Perishable-Prediction\Datasets\weather_data.csv")
weekly_sales_data = pd.read_csv(r"C:\Users\Etteba\OneDrive\Documents\DataSciencePortfolioProjects\PredPerishProducts\Amdari-Perishable-Prediction\Datasets\weekly_sales.csv")

In [6]:
#Inspect the datasets
print(prod_data.head(5))
print(store_data.head(5))    
print(supplier_data.head(5))
print(weather_data.head(5))
print(weekly_sales_data.head(5))


   Product_ID              Product_Name Product_Category  Shelf_Life_Days  \
0        1000    Whole Wheat Bread 800g           Bakery                4   
1        1001       White Sandwich Loaf           Bakery                2   
2        1002          Croissant 4-pack           Bakery                2   
3        1003  Blueberry Muffins 6-pack           Bakery                4   
4        1004    Chocolate Chip Cookies           Bakery                4   

   Supplier_ID  
0            4  
1            8  
2            5  
3           10  
4            7  
   Store_ID      Region  Store_Size  Cold_Storage_Capacity
0       500      London       12000                   3788
1       501    Midlands        5000                   1086
2       502  North East        4000                    998
3       503    Midlands        5000                   1243
4       504      London       10000                   3330
   Supplier_ID              Supplier_Name  Lead_Time_Days  Supply_Capacity
0     

In [7]:
# Checkl the columns and data types of each dataset
print(prod_data.info())
print(store_data.info())
print(supplier_data.info())
print(weather_data.info())  
print(weekly_sales_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Product_ID        32 non-null     int64 
 1   Product_Name      32 non-null     object
 2   Product_Category  32 non-null     object
 3   Shelf_Life_Days   32 non-null     int64 
 4   Supplier_ID       32 non-null     int64 
dtypes: int64(3), object(2)
memory usage: 1.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Store_ID               15 non-null     int64 
 1   Region                 15 non-null     object
 2   Store_Size             15 non-null     int64 
 3   Cold_Storage_Capacity  15 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 612.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 

In [8]:
# Check the column names to see if there are any common columns to merge on
print(prod_data.columns)
print(store_data.columns)
print(supplier_data.columns)
print(weather_data.columns)
print(weekly_sales_data.columns)

Index(['Product_ID', 'Product_Name', 'Product_Category', 'Shelf_Life_Days',
       'Supplier_ID'],
      dtype='object')
Index(['Store_ID', 'Region', 'Store_Size', 'Cold_Storage_Capacity'], dtype='object')
Index(['Supplier_ID', 'Supplier_Name', 'Lead_Time_Days', 'Supply_Capacity'], dtype='object')
Index(['Week_Number', 'Region', 'Avg_Temperature', 'Rainfall', 'Holiday_Flag'], dtype='object')
Index(['Week_Number', 'Product_ID', 'Store_ID', 'Units_Sold',
       'Marketing_Spend', 'Discount_Percent', 'Wastage_Units', 'Price'],
      dtype='object')


The result above shows that there are four common columns or keys we can use to merge the datasets: Product_ID, Supplier_ID, Region and Week_Number.

Hence we will merge the datasets logically in this order:

* step 1: Merge Weekly Sales with the Product Details via Product_ID to get a new dataset called merged_df
* step 2: merged_df would be merged with supplier_info (via Supplier_ID)
* step 3: merged_df with store_info (via Store_ID)
* step 4: merged_df with with weather_data (via Week_Number and Region) 


In [ ]:
# --- Merge datasets logically ---

# Step 1: Merge Weekly Sales with Product Details
merged_df = weekly_sales_data.merge(prod_data, on='Product_ID', how='left')

# Step 2: Merge with Supplier Info (via Supplier_ID)
merged_df = merged_df.merge(supplier_data, on='Supplier_ID', how='left')

# Step 3: Merge with Store Info (via Store_ID)
merged_df = merged_df.merge(store_data, on='Store_ID', how='left')

# Step 4: Merge with Weather Data (via Week_Number and Region)
merged_df = merged_df.merge(weather_data, on=['Week_Number', 'Region'], how='left')

# Lets inspect the first 10 rows of the final merged dataset
print(merged_df.head(10))


  Week_Number  Product_ID  Store_ID  Units_Sold  Marketing_Spend  \
0    2024-W01        1000       500        4853           670.37   
1    2024-W01        1001       500        4274          1089.62   
2    2024-W01        1002       500        4215          1004.99   
3    2024-W01        1003       500        5768           859.45   
4    2024-W01        1004       500        3403           576.70   
5    2024-W01        1005       500        3757           937.48   
6    2024-W01        1006       500        5562           546.51   
7    2024-W01        1007       500        2957           984.57   
8    2024-W01        1008       500        5925           641.34   
9    2024-W01        1009       500        7494           853.40   

   Discount_Percent  Wastage_Units  Price              Product_Name  \
0                 0            718   2.46    Whole Wheat Bread 800g   
1                 0            641   1.27       White Sandwich Loaf   
2                10            632   2

In [ ]:
# --- Export to CSV ---
output_path = "merged_dataset.csv"
merged_df.to_csv(output_path, index=False)